In [23]:
import pandas as pd
import numpy as np
import math
import json
import yaml
import operator

from itertools import combinations
from collections import Counter as ctr

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet

In [8]:
idf_df = pd.read_csv('idf.csv')

In [9]:
idf_df.head()

,Unnamed: 0,word,idf
0,0,cowhide,14.515398
1,1,porcupine,10.784031
2,2,untruthfully,15.964705
3,3,rebooting,14.141583
4,4,purcell,10.627598


In [10]:
highest_count_word_df = pd.read_csv('max_d.csv')
highest_count_word_df = highest_count_word_df.rename(columns={'Unnamed: 0': 'DocID'})

#highest_count_word_df[highest_count_word_df.DocID == 17809]
highest_count_word_df.head()

,DocID,max_d
0,0,11
1,1,3
2,2,2
3,3,7
4,4,5


In [11]:
df_index = pd.read_csv('index.csv', names=['word', 'frequency'])

In [12]:
df_index.head()

,word,frequency
0,cowhide,"{29104: 1, 82126: 1, 90797: 9, 108214: 1, 1287..."
1,porcupine,"{17809: 2, 19610: 1, 20467: 2, 25916: 12, 2815..."
2,untruthfully,"{53136: 1, 123616: 1, 128567: 3, 249592: 1, 25..."
3,rebooting,"{29971: 1, 58410: 1, 61198: 1, 64833: 1, 96158..."
4,purcell,"{570: 1, 5495: 2, 17002: 1, 17676: 1, 18439: 5..."


In [80]:
ws = ['this', 'is', 'a', 'test']

co = combinations(ws, len(ws)-1)

# for comb in co:
#     print(list(comb))


In [13]:
lemmatizer = WordNetLemmatizer()
stops = stopwords.words('english')

def clean(doc):
    doc_low = doc.lower().replace("–", " ")
    words = word_tokenize(doc_low)
    words = [lemmatizer.lemmatize(w).strip() for w in words if not w in stops and wordnet.synsets(w) and w.isalpha()]
    return words

In [14]:
#THERE HAS TO BE AT LEAST TWO NONSTOP LEMMATIZED WORDS IN THE QUERY FOR THIS TO WORK
#IDEALLY THERE IS 3+ WORDS
def get_candidate_resources(query):
    clean_query = clean(query)
    CR = []
    all_words_dict = []
    for w in clean_query:
        w_dict = yaml.load(df_index.loc[df_index.word==w, 'frequency'].item())
        all_words_dict.append(list(w_dict.keys()))
    CR.extend(set(all_words_dict[0]).intersection(*all_words_dict[1:]))
    if len(CR)!=50:
        combs = combinations(clean_query, len(query)-1)
        for comb in list(combs):
            all_words_dict = []
            for w in list(comb):
                w_dict = yaml.load(df_index.loc[df_index.word==w, 'frequency'].item())
                all_words_dict.append(list(w_dict.keys()))
            CR.extend(set(all_words_dict[0]).intersection(*all_words_dict[1:]))
            if len(set(CR))<50:
                continue
            elif len(set(CR))>50:
                CR = set(CR)[:49]
                break
            else: break
    return CR

In [15]:
def idf(w):
    idf_value = idf_df.loc[idf_df.word==w, 'idf'].item()
    return int(idf_value)

In [16]:
def frequency(w,d):
    word_freq_dict = yaml.load(df_index.loc[df_index.word==w, 'frequency'].item())
    word_freq = word_freq_dict[d]
    return int(word_freq)

In [17]:
def term_freq(w,d):
    max_showing_word = int(highest_count_word_df.loc[highest_count_word_df.DocID==d, 'max_d'].item())
    return frequency(w,d)/max_showing_word

In [18]:
def relevance_ranking(q, d):
    clean_query = clean(q)
    rel_score = 0
    for word in clean_query:
        result = term_freq(word,d) * idf(word)
        rel_score += result
    return rel_score

In [19]:
q = "Beautiful dogs everywhere love them all"
r = get_candidate_resources(q)

In [20]:
rel_docs = {}
for resource in r:
    rel_docs[resource] = relevance_ranking(q,resource)

In [21]:
rel_docs

{75648: 5.333333333333333,
 140289: 0.3116883116883117,
 322691: 0.0687593423019432,
 1331468: 0.5842696629213483,
 125710: 0.37373737373737376,
 365198: 0.2764227642276423,
 380558: 0.5901639344262295,
 192658: 0.05156537753222837,
 786323: 0.4137931034482759,
 494356: 1.2105263157894737,
 979220: 2.0,
 253082: 1.2413793103448276,
 115354: 0.5291005291005291,
 1035935: 0.20702402957486135,
 1584927: 0.6557377049180328,
 1108903: 0.3446054750402576,
 162089: 0.5106382978723404,
 718505: 0.13389121338912133,
 548907: 0.5918367346938775,
 807849: 0.5517241379310345,
 1543031: 1.2743362831858407,
 132528: 0.8354430379746836,
 770993: 0.5660377358490566,
 1204402: 1.0444444444444443,
 468148: 2.4,
 814139: 0.2809917355371901,
 340414: 0.9090909090909092,
 509886: 0.851063829787234,
 1082946: 1.2761904761904763,
 822980: 0.5846153846153846,
 495942: 1.0,
 688970: 1.12,
 225873: 1.0632911392405062,
 262353: 2.4615384615384617,
 202076: 0.4482758620689655,
 377438: 0.6611570247933884,
 209761

In [29]:
sorted_rel_docs = sorted(rel_docs.items(), key=operator.itemgetter(1), reverse=True)

In [30]:
sorted_rel_docs

[(75648, 5.333333333333333),
 (1658594, 4.4),
 (1077479, 4.142857142857142),
 (262353, 2.4615384615384617),
 (468148, 2.4),
 (979220, 2.0),
 (1411829, 1.9166666666666665),
 (1082946, 1.2761904761904763),
 (1543031, 1.2743362831858407),
 (253082, 1.2413793103448276),
 (494356, 1.2105263157894737),
 (514418, 1.1794871794871795),
 (361079, 1.1666666666666665),
 (688970, 1.12),
 (225873, 1.0632911392405062),
 (1204402, 1.0444444444444443),
 (392048, 1.0144927536231885),
 (495942, 1.0),
 (340414, 0.9090909090909092),
 (509886, 0.851063829787234),
 (132528, 0.8354430379746836),
 (377438, 0.6611570247933884),
 (1584927, 0.6557377049180328),
 (548907, 0.5918367346938775),
 (380558, 0.5901639344262295),
 (822980, 0.5846153846153846),
 (1331468, 0.5842696629213483),
 (770993, 0.5660377358490566),
 (807849, 0.5517241379310345),
 (135138, 0.5303867403314917),
 (115354, 0.5291005291005291),
 (162089, 0.5106382978723404),
 (202076, 0.4482758620689655),
 (786323, 0.4137931034482759),
 (125710, 0.3737

In [37]:
results = [i[0] for i in sorted_rel_docs[:5]]

In [38]:
results

[75648, 1658594, 1077479, 262353, 468148]